# Assignment 3
## Group name: ID2214-10
### Project members: 
[Ruben van Gemeren, rubenvg@kth.se]

[Johan Edeland, johede@kth.se]

[Samuel Horácek, horacek@kth.se]

### Declaration:
By submitting this assignment, it is hereby declared that all group members listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214), no part of the solution has been provided by someone not listed as a project member above, and no part of the solution has been generated by a system.

It is furthermore declared that the submitted assignment will not be shared during the course, with any individual other than the group members listed above and teachers of the course ID2214/FID3214. In particular, the assignment will not be uploaded to any public repository. The submitted assignment can be shared after the course only if written consent has been provided by the course responsible of ID2214/FID3214.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import pandas as pd
import time
import sklearn
from sklearn.tree import DecisionTreeClassifier

In [3]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"sklearn version: {sklearn.__version__}")

Python version: 3.11.9
NumPy version: 2.1.3
Pandas version: 2.2.3
sklearn version: 1.5.2


## Reused functions from Assignment 1

In [4]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
# For AUC
from typing import Counter

def create_column_filter(df):
    df_copy = df.copy()
    column_filter = []

    for column in df_copy.columns:
        if column == "CLASS" or column == "ID":
            column_filter.append(column)
        else:
            unique_values = df_copy[column].dropna().unique()
            if len(unique_values) > 1:
                column_filter.append(column)
            else:
                df_copy.drop(column, axis=1, inplace=True)

    return df_copy, column_filter

def apply_column_filter(df, column_filter):
    return df[column_filter]


# Hint 2: Consider using fillna

# Implement the functions create_imputation and apply_imputation here:

def create_imputation(df):
    df_copy = df.copy()
    imputation = {}

    for column in df_copy.columns:
        if column == "CLASS" or column == "ID":
            continue
        if df_copy[column].isnull().all():
            if df_copy[column].dtype == "float" or df_copy[column].dtype == "int":
                df_copy[column].fillna(0, inplace=True)
                imputation[column] = 0
            elif df_copy[column].dtype == "object":
                df_copy[column].fillna("", inplace=True)
                imputation[column] = ""
            elif df_copy[column].dtype.name == "category":
                df_copy[column].fillna(df_copy[column].cat.categories[0], inplace=True)
                imputation[column] = df_copy[column].cat.categories[0]
        if df_copy[column].dtype == "float" or df_copy[column].dtype == "int":
            mean = df_copy[column].mean()
            df_copy[column].fillna(mean, inplace=True)
            imputation[column] = mean
        elif df_copy[column].dtype == "object" or df_copy[column].dtype.name == "category":
            mode = df_copy[column].mode()[0]
            df_copy[column].fillna(mode, inplace=True)
            imputation[column] = mode

    return df_copy, imputation

def apply_imputation(df, imputation):
    df_copy = df.copy()

    for column in df_copy.columns:
        if column in imputation:
            df_copy[column].fillna(imputation[column], inplace=True)

    return df_copy


def create_one_hot(df):
    df_copy = df.copy()
    one_hot = {}

    for column in df_copy.columns:
        if column == "CLASS" or column == "ID":
            continue
        if df_copy[column].dtype == "object" or df_copy[column].dtype.name == "category":
            categories = df_copy[column].unique()
            for category in categories:
                new_column_name = column + "_" + str(category)
                df_copy[new_column_name] = (df_copy[column] == category).astype("float")
            df_copy.drop(column, axis=1, inplace=True)
            one_hot[column] = categories

    return df_copy, one_hot

def apply_one_hot(df, one_hot):
    df_copy = df.copy()

    for column in df_copy.columns:
        if column in one_hot:
            categories = one_hot[column]
            for category in categories:
                new_column_name = column + "_" + str(category)
                df_copy[new_column_name] = (df_copy[column] == category).astype("float")
            df_copy.drop(column, axis=1, inplace=True)

    return df_copy

def accuracy(df, correctlabels):
    correct = 0

    for i, row in df.iterrows():
        # Returns the first occurance of the maximum value, so the former case from the hint
        predicted_label = row.idxmax()
        if predicted_label == correctlabels[i]:
            correct += 1

    return correct / len(correctlabels)

def brier_score(df, correctlabels):
    brier_score = 0

    for i, row in df.iterrows():
        correct_index = np.where(df.columns == correctlabels[i])[0][0]
        correct_vector = np.zeros(len(row))
        correct_vector[correct_index] = 1
        brier_score += np.sum((row - correct_vector) ** 2)

    return brier_score / len(correctlabels)

def auc(df, correctlabels):
    class_labels = df.columns
    n = len(correctlabels)
    class_counts = Counter(correctlabels)

    class_freq = {c: class_counts[c] / n for c in class_labels}

    auc_total = 0

    for c in class_labels:
        scores = df[c]
        is_class = [1 if label == c else 0 for label in correctlabels]

        # print (is_class)

        # Create DataFrame for scores and correct labels, then sort by scores
        score_df = pd.DataFrame({'score': scores, 'is_class': is_class})
        score_df = score_df.sort_values(by='score', ascending=False)

        tp, fp = 0, 0
        auc_class = 0
        tpr_prev, fpr_prev = 0, 0

        pos_total = class_counts[c]
        neg_total = n - pos_total

        # Calculate TPR and FPR for each score threshold
        for _, row in score_df.iterrows():
            if row['is_class'] == 1:
                tp += 1
            else:
                fp += 1

            tpr = tp / pos_total if pos_total > 0 else 0
            fpr = fp / neg_total if neg_total > 0 else 0

            # Accumulate AUC using trapezoidal rule
            auc_class += (fpr - fpr_prev) * (tpr + tpr_prev) / 2
            tpr_prev, fpr_prev = tpr, fpr

        # Weighted AUC by class frequency
        auc_total += class_freq[c] * auc_class

    return auc_total

## 1. Define the class RandomForest

In [5]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment.


class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees=100):
        df_copy, self.column_filter = create_column_filter(df)
        df_copy, self.imputation = create_imputation(df_copy)
        df_copy, self.one_hot = create_one_hot(df_copy)

        self.labels = sorted(df["CLASS"].unique())

        X = apply_one_hot(apply_imputation(apply_column_filter(df, self.column_filter), self.imputation), self.one_hot)
        X.drop(["CLASS"], axis=1, inplace=True)

        y = df["CLASS"]

        self.model = []

        for _ in range(no_trees):
            tree = DecisionTreeClassifier(max_features="log2")
            sample = np.random.choice(len(X), len(X), replace=True)
            tree.fit(X.iloc[sample], y.iloc[sample])
            self.model.append(tree)

    def predict(self, df):
        X = apply_one_hot(apply_imputation(apply_column_filter(df, self.column_filter), self.imputation), self.one_hot)
        X.drop(["CLASS"], axis=1, inplace=True)

        predictions = np.zeros((len(X), len(self.labels)))

        for tree in self.model:
            predictions += tree.predict_proba(X)

        predictions /= len(self.model)

        return pd.DataFrame(predictions, columns=self.labels)



In [6]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.35 s.
Testing time: 0.16 s.
Accuracy: 0.9248
AUC: 0.9916
Brier score: 0.1769


In [7]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0213


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [8]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 


class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees=100):
        df_copy, self.column_filter = create_column_filter(df)
        df_copy, self.imputation = create_imputation(df_copy)
        df_copy, self.one_hot = create_one_hot(df_copy)

        self.labels = sorted(df["CLASS"].unique())

        X = apply_one_hot(apply_imputation(apply_column_filter(df, self.column_filter), self.imputation), self.one_hot)
        X.drop(["CLASS"], axis=1, inplace=True)

        y = df["CLASS"]

        self.model = []

        for _ in range(no_trees):
            tree = DecisionTreeClassifier(max_features="log2")
            sample = np.random.choice(len(X), len(X), replace=True)
            tree.fit(X.iloc[sample], y.iloc[sample])
            self.model.append(tree)

    def predict(self, df):
        X = apply_one_hot(apply_imputation(apply_column_filter(df, self.column_filter), self.imputation), self.one_hot)
        X.drop(["CLASS"], axis=1, inplace=True)

        predictions = np.zeros((len(X), len(self.labels)))

        for tree in self.model:
            tree_predictions = tree.predict_proba(X)
            tree_labels = tree.classes_

            # Could be optimized, as per the hints
            for idx, label in enumerate(tree_labels):
                label_index = self.labels.index(label)
                predictions[:, label_index] += tree_predictions[:, idx]

        predictions /= len(self.model)

        return pd.DataFrame(predictions, columns=self.labels)


In [9]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.36 s.
Testing time: 0.17 s.
Accuracy: 0.9488
AUC: 0.9732
Brier score: 0.1011


## 2b. Estimate predictive performance using out-of-bag predictions

In [10]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively. By "aggregated out-of-bag predictions" is here meant the sum of all 
#         predicted probabilities (one sum per class and instance). These sums should be divided by the number of predictions
#         (stored in the vector) in order to obtain a single class probability distribution per training instance. 
#         This distribution is considered to be the out-of-bag prediction for each instance, and e.g., the class that 
#         receives the highest probability for each instance can be compared to the correct label of the instance, 
#         when calculating the accuracy using the out-of-bag predictions.
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector
#
#         For example, assuming that we have two class labels, then we may end up with the following matrix:
#
#         2 4
#         4 4
#         5 0
#         ...
#
#         and the vector (no. of predictions) (6, 8, 5, ...)
#
#         The resulting class probability distributions are:
#
#         0.333... 0.666...
#         0.5 0.5
#         1.0 0

class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.oob_acc = None

    def fit(self, df, no_trees=100):
        df_copy, self.column_filter = create_column_filter(df)
        df_copy, self.imputation = create_imputation(df_copy)
        df_copy, self.one_hot = create_one_hot(df_copy)

        self.labels = sorted(df["CLASS"].unique())

        X = apply_one_hot(apply_imputation(apply_column_filter(df, self.column_filter), self.imputation), self.one_hot)
        X.drop(["CLASS"], axis=1, inplace=True)

        y = df["CLASS"]

        self.model = []
        oob_predictions = np.zeros((len(X), len(self.labels)))
        oob_counts = np.zeros(len(X))

        for _ in range(no_trees):
            tree = DecisionTreeClassifier(max_features="log2")
            sample = np.random.choice(len(X), len(X), replace=True)
            tree.fit(X.iloc[sample], y.iloc[sample])
            self.model.append(tree)

            # Instances (indexes) not included in the bootstrap sample
            oob = np.setdiff1d(np.arange(len(X)), sample)
            tree_predictions = tree.predict_proba(X.iloc[oob])
            tree_labels = tree.classes_

            for idx, label in enumerate(tree_labels):
                label_index = self.labels.index(label)
                oob_predictions[oob, label_index] += tree_predictions[:, idx]

            oob_counts[oob] += 1

        oob_predictions /= oob_counts[:, None]

        self.oob_acc = accuracy(pd.DataFrame(oob_predictions, columns=self.labels), y)

    def predict(self, df):
        X = apply_one_hot(apply_imputation(apply_column_filter(df, self.column_filter), self.imputation), self.one_hot)
        X.drop(["CLASS"], axis=1, inplace=True)

        predictions = np.zeros((len(X), len(self.labels)))

        for tree in self.model:
            tree_predictions = tree.predict_proba(X)
            tree_labels = tree.classes_

            for idx, label in enumerate(tree_labels):
                label_index = self.labels.index(label)
                predictions[:, label_index] += tree_predictions[:, idx]

        predictions /= len(self.model)

        return pd.DataFrame(predictions, columns=self.labels)

In [11]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.57 s.
OOB accuracy: 0.9577
Testing time: 0.17 s.
Accuracy: 0.9510
AUC: 0.9675
Brier score: 0.1021


In [12]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.